In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import numpy as np 
import pathlib

# Importing utitility functions for training
from PT_files.model import DnCNN, DnCNN_B
from PT_files.Dataset import Img_Dataset, Large_Img_Dataset
import PT_files.preprocess_data as ppd
import PT_files.save_load as sl
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

Using cuda device


In [2]:
data_6k = sl.NERSC_load('test_data70-6000.npy')

In [3]:
def grid_window(dataset,
                model,
                model_params,
                samp_idx,
                h_start,
                h_end,
                w_start,
                w_end):
    
    full = np.empty((1, 1, 6000, 6000))
    count = np.empty((1, 1, 6000, 6000))
    
    noise_data = dataset[0]
    params_name = model_params
    
    current_dir = pathlib.Path().resolve()
    model_params_path = current_dir / 'Model_params'
    assert model_params_path.exists()
    model_path = model_params_path / params_name
    
    model = model()
    model.to(device)
    model.load_state_dict(torch.load(str(model_path)))
    model.eval()
    
    with torch.no_grad():
        
        torch.cuda.empty_cache()
        test_noise = torch.as_tensor(noise_data[samp_idx:samp_idx+1, :, h_start:h_end, w_start:w_end])
        test_noise = test_noise.to(device)
        
        output = model(test_noise)
        resid_img = output.cpu().detach().numpy()
        
        full[:, :, h_start:h_end, w_start:w_end] += resid_img
        count[:, :, h_start:h_end, w_start:w_end] += 1
        
    return full, count

In [4]:
full, count = full_grid_pass_window(dataset=data_6k,
                             model= DnCNN,
                             samp_idx=0,
                             w_start=0,
                             w_end=2000,
                             h_start=0,
                             h_end=2000)

In [ ]:
def grid_window(dataset,
                model,
                model_params,
                samp_idx,
                h_start,
                h_end,
                w_start,
                w_end):
    
    full = np.empty((1, 1, 6000, 6000))
    count = np.empty((1, 1, 6000, 6000))
    
    noise_data = dataset[0]
    params_name = model_params
    
    current_dir = pathlib.Path().resolve()
    model_params_path = current_dir / 'Model_params'
    assert model_params_path.exists()
    model_path = model_params_path / params_name
    
    model = model()
    model.to(device)
    model.load_state_dict(torch.load(str(model_path)))
    model.eval()
    
    with torch.no_grad():
        
        torch.cuda.empty_cache()
        test_noise = torch.as_tensor(noise_data[samp_idx:samp_idx+1, :, h_start:h_end, w_start:w_end])
        test_noise = test_noise.to(device)
        
        output = model(test_noise)
        resid_img = output.cpu().detach().numpy()
        
        full[:, :, h_start:h_end, w_start:w_end] += resid_img
        count[:, :, h_start:h_end, w_start:w_end] += 1
        
    return full, count

In [ ]:
def full_img_pass(dataset,
                  model,
                  model_params,
                  model_size: int,
                  samp_idx: int):
    
    noise_data = dataset[0]    
    patch_length = model_size
    num_patch_across = int(len(noise_data[0][0]) / patch_length)
    patch_end_idxs = []
    for i in range(patch_length):
        patch_end_idxs.append(patch_length*(i+1))
        
    for j in range(len(patch_end_idxs)):
        for k in range(len(patch_end_idxs)):

            full, count = grid_window(dataset=dataset,
                                      model=model,
                                      model_params=model_params,
                                      samp_idx=samp_idx,
                                      h_start=0,
                                      h_end=patch_end_idxs[j],
                                      w_start=0,
                                      w_end=patch_end_idxs[k])
    
    return full, count